In [1]:
import datasets
from collections import defaultdict

# Dataloading step
#! Trust remote code sounds a bit unwise but i guess i have to trust
pre_procress_dataset = datasets.load_dataset('ptb_text_only', split='train', trust_remote_code=True)

/home/chris/haw/4semester/nlp/lab03/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Processing step:

def add_stop_token_batch(batch, sentence_column, stop_token="<stop>"):
    """Adds stop token to input dataset"""
    new_texts = [sentence + ' ' + stop_token for sentence in batch[sentence_column]]

    return {sentence_column: new_texts}

def filter_short_sentence_batch(batch, sentence_column, min_words: int=3):
    """Filters out any sentence with less than min_words"""
    new_text = [len(sentence.split()) >= min_words for sentence in batch[sentence_column]]
    return new_text


dataset_stop_token = pre_procress_dataset.map(
    add_stop_token_batch,
    batched=True,
    fn_kwargs={'sentence_column': 'sentence'} 
)

dataset = dataset_stop_token.filter(
    filter_short_sentence_batch,
    batched=True,
    fn_kwargs={'sentence_column': 'sentence'}
)

dataset_split = dataset.train_test_split(test_size=0.2, shuffle=True)

In [ ]:
def get_rares(threshold):
    """
    This is much better than the suggested way in the assignment,
    The assignment didnt make sense at all, why the fuck should i 
    implement a remove_rares functuin that takes in two datasets?
    """
    counter_dict = defaultdict(int)
    for sentence in dataset_split['train']['sentence']:
        for word in sentence.split():
                counter_dict[word] += 1

    return {word for word, count in counter_dict.items() if count < threshold}


def replace_rares_batch(batch, rares):
    """Replaces words present in rares with '<unk>'."""
    new_sentences = []
    for sentence in batch['sentence']:
        new_sentence_parts = []
        for word in sentence.split():
            if word in rares:
                new_sentence_parts.append('<unk>')
            else:
                new_sentence_parts.append(word)
        new_sentences.append(' '.join(new_sentence_parts))
    return {'sentence': new_sentences}


In [4]:
print(dataset_split)
for i in range(5):
    print(dataset_split['train'][i]['sentence'])

DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 33544
    })
    test: Dataset({
        features: ['sentence'],
        num_rows: 8387
    })
})
it was worth it just for the look on albert 's face <stop>
translation motorola appears to have taken a hitachi technology that is <unk> in the u.s. hitachi says and tried to make it look like a new technology <stop>
<unk> <unk> N years old has lived in the district most of her life <stop>
so far though there have been no allegations that the contributions the ldp members received amounted to <unk> <stop>
volume N shares <stop>


In [5]:

def estimate_unigram(train_set):
    """
    Basic unigram model that simply counts all words
    and returns them as keys with there probalbility as values
    """
    total_words = 0
    model_dict = defaultdict(int) #! i didnt know that int() returns 0 this could be good to keep in minde BUT HERE WE NEED INT WIHTOUT () THIS IS VERY CONFUSING!!!
    model = {}
    for sentence in train_set:
        for word in sentence.split():
            model_dict[word] += 1
            total_words += 1
    
    for key in model_dict.keys():
        model[key] = (model_dict[key] / total_words)
    
    return model

In [6]:
import math

def unigram_sentence_logp(sentence, model):
    """ 
    This is the corss-entropy formula from the lecture/assignment,
    we look at each word of a sentence and calculate the log
    and add them up to get the probability of the sentence.
    If we dont know a word -> Prob = 0. the log would be -inf!
    BUT per formula we take the negativ log so it would become inf!
    BUT this does not feel good to me, i would rather still return the negativ inf.
    Not Shure if returning this is what we should do here, but thats the way i like it.
    """
    total_log = 0.0 #! Dont forget that we need float here!

    for word in sentence.split():
        if word in model:
            total_log += -(math.log2(model[word])) #! I use -log2 here because we use -log2 in the cross-entropy formula
        else:
            return float('-inf')
    
    return total_log


In [7]:
model = estimate_unigram(dataset_split['train']['sentence'])

In [8]:
print(unigram_sentence_logp("the the the <stop>", model))
print(unigram_sentence_logp("i love computer science <stop>", model))

17.067479363498016
55.44314259138373


In [9]:
def unigram_perplexity(dataset, model):
    """
    This is the Perplexity formula from the lecture/assignment.
    We now add up the log2 of each sentence in the dataset(test set)
    and then we just divide the log with the number of words (cross-entropy).
    Then we calculate and return the perpelecity by raising 2 to the power of the cross-entropy
    """
    dataset_log = 0.0
    dataset_words = 0

    for sentence in dataset['sentence']:
        sentence_log = unigram_sentence_logp(sentence, model)
        if sentence_log == float('-inf'):
            continue
        dataset_log += sentence_log
        dataset_words += len(sentence)
    
    if dataset_words == 0:
        raise Exception("There are no valid sentences!")
    
    print('Dataset Log:',dataset_log)
    print('Words:',dataset_words)
    h = dataset_log / dataset_words
    perplexity = 2**h

    return perplexity

In [10]:
print(unigram_perplexity(dataset_split['test'], model))

Dataset Log: 1763984.8268081625
Words: 1060126
3.1688049063181993


In [19]:
rares = get_rares(5)

dataset_split['train'] = dataset_split['train'].map(
    replace_rares_batch,
    batched=True,
    fn_kwargs={'rares_set': rares}
)

dataset_split['test'] = dataset_split['test'].map(
    replace_rares_batch,
    batched=True,
    fn_kwargs={'rares_set': rares} 
)


Map: 100%|██████████| 8387/8387 [00:00<00:00, 376207.17 examples/s]


In [20]:
model = estimate_unigram(dataset_split['train']['sentence'])

In [21]:
print(unigram_perplexity(dataset_split['test'], model))

Dataset Log: 1737400.6763911191
Words: 1055104
3.1310856219188477
